In [1]:
a = 1
b = 2
print(a+b)

3


In [2]:
!pip install sympy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 11.3 MB/s eta 0:00:0000:01
You should consider upgrading via the '/Users/exidea258/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
import sympy
z=sympy.pi.evalf(100)
print(z)

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117068


In [4]:
import numpy as np
import requests
from requests.exceptions import Timeout
from bs4 import BeautifulSoup
import io
import re

In [ ]:
urls = []
pre_extracted_urls = []
pre_extracted_urls.append('https://www.bbc.com/news')
for depth in range(2):
    print('start depth: '+str(depth)+'.........................')
    extracted_urls = []
    for i in range(len(pre_extracted_urls)):
        try:
            res = requests.get(pre_extracted_urls[i], timeout=3.0)
        except Timeout:
            print('Connection timeout')
            continue
        soup = BeautifulSoup(res.text, 'html.parser')
        elems = soup.find_all(href=re.compile("/news/"))
        print(str(len(elems))+' URLs extracted'+'('+str(i+1)+'/'+str(len(pre_extracted_urls))+')')
        for j in range(len(elems)):
            url = elems[j].attrs['href']
            if not 'http' in url:
                extracted_urls.append('https://www.bbc.com'+url)
            else:
                extracted_urls.append(url)
    extracted_urls = np.unique(extracted_urls).tolist()
    pre_extracted_urls = extracted_urls
    urls.extend(extracted_urls)
    urls = np.unique(urls).tolist()
    print('total: '+str(len(urls))+' URLs')
print('start extracting html....')
texts = []
for i in range(len(urls)):
    try:
        res = requests.get(urls[i], timeout=3.0)
    except Timeout:
        print('Connection timeout')
        continue
    soup = BeautifulSoup(res.text, "html.parser")
    elems = soup.select('#page > div > div.container > div > div.column--primary > div.story-body > div.story-body__inner > p')
    if not len(elems) == 0:
        for j in range(len(elems)):
            texts.append(str(elems[j]))
    print(str(i+1)+' / '+str(len(urls))+' finished')
text = ' '.join(texts)
p = re.compile(r"<[^>]*?>")
text = p.sub("", text)
text = re.sub('["“”,—]','', text)
text = text.lower()
text = text.replace('. ','\n')
with io.open('articles_bbc2.txt', 'w', encoding='utf-8') as f:
    f.write(text)
with io.open('articles_bbc.txt', 'w', encoding='utf-8') as f:
    f.write(text)